In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
import pickle

In [18]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
label_conder_gender=LabelEncoder()
df['Gender']=label_conder_gender.fit_transform(df['Gender'])

In [21]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [22]:
one_hot_encoder=OneHotEncoder(handle_unknown='ignore')

geo_encoder=one_hot_encoder.fit_transform(df[['Geography']]).toarray()


In [23]:
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [24]:
geo_encoder_df=pd.DataFrame(geo_encoder,columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [25]:
df=pd.concat([df,geo_encoder_df],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [26]:
df=df.drop(['Geography'],axis=1)

In [27]:
X=df.drop('EstimatedSalary',axis=1)
y=df['EstimatedSalary']

In [28]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [29]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [30]:
with open('label_conder_gender2.pkl','wb') as file:
    pickle.dump(label_conder_gender,file)

with open('one_hot_encoder2.pkl','wb') as file:
    pickle.dump(one_hot_encoder,file)

with open('scaler2.pkl','wb') as file:
    pickle.dump(scaler,file)
    

# ANN Regression

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [32]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [34]:
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [35]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 100367.6875 - mae: 100367.6875 - val_loss: 98480.3047 - val_mae: 98480.3047
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 99501.4531 - mae: 99501.4531 - val_loss: 96748.0234 - val_mae: 96748.0234
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 96509.2188 - mae: 96509.2188 - val_loss: 92383.5312 - val_mae: 92383.5312
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 90703.0156 - mae: 90703.0156 - val_loss: 85215.0469 - val_mae: 85215.0469
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 82393.0391 - mae: 82393.0391 - val_loss: 76152.3828 - val_mae: 76152.3828
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 72893.2734 - mae: 72893.2734 - val_loss: 67008.5234 - val_mae: 67008.5234
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 64043.4375 - mae: 64043.437

In [36]:
%load_ext tensorboard

In [38]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 21884), started 0:00:03 ago. (Use '!kill 21884' to kill it.)

In [39]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 0s 1ms/step - loss: 50292.3789 - mae: 50292.3789
Test MAE: 50292.37890625


In [40]:
model.save('regression_model.h5')

n:\PW\udamy\End to End deep learning project using ANN\Code\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
